In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import curves.bezier as bezier
import curves.fitCurves as fit
import curves.fitArcs as fitArcs
import StressTools as tools
import utils
import fitting
from joblib import Parallel, delayed, Memory
import StressEquations as simon
from numba import jit

interior = utils.import_interior('interior1')

TOLERANCE = 1

delphi = pd.read_csv("./obsData/DelphiLonLatAT.txt", header=None, sep=' ', names=['lon', 'lat'])
delphi = delphi.sort_values(['lon', 'lat'])

arcs = [
    delphi[0:9],
    delphi[9:18],
    delphi[18:27],
    delphi[27:34],
    delphi[34:]
]

ModuleNotFoundError: No module named 'curves.fitArcs'

In [11]:
@jit(nopython=True)
def stressththe(constA, constB, e, oblq, phase, colat, lon, he, le):

    beta20ththe = 0.75*(3.*he - 10.*le)*np.cos(2.*colat) + 0.75*(he - 2.*le)
    beta21ththe = 1.5*(3.*he - 10.*le)*np.sin(2.*colat)
    beta22ththe = -1.5*(3.*he - 10.*le)*np.cos(2.*colat) + 4.5*(he - 2.*le)

    ththe = constB*(-6.*e*beta20ththe*np.cos(constA) + e*beta22ththe*(4.*np.sin(2.*lon)*np.sin(constA)+3. *
                                                                      np.cos(2.*lon)*np.cos(constA)) + 4.*np.cos(oblq)*np.sin(oblq)*beta21ththe*np.cos(lon)*np.sin(phase + constA))

    return ththe

@jit(nopython=True)
def stressphphe(constA, constB, e, oblq, phase, colat, lon, he, le):

    beta20phphe = 0.75*(3.*he - 8.*le)*np.cos(2.*colat) + 0.75*(he - 4.*le)
    beta21phphe = 1.5*(3.*he - 8.*le)*np.sin(2.*colat)
    beta22phphe = -1.5*(3.*he - 8.*le)*np.cos(2.*colat) + 4.5*(he - 4.*le)

    phphe = constB*(-6.*e*beta20phphe*np.cos(constA) + e*beta22phphe*(4.*np.sin(2.*lon)*np.sin(constA)+3. *
                                                                      np.cos(2.*lon)*np.cos(constA)) + 4.*np.cos(oblq)*np.sin(oblq)*beta21phphe*np.cos(lon)*np.sin(phase + constA))

    return phphe

@jit(nopython=True)
def stressthphe(constA, constB, e, oblq, phase, colat, lon, he, le):

    beta21thphe = 3.*le*np.sin(colat)
    beta22thphe = 3.*le*np.cos(colat)

    thphe = constB*(2.*e*beta22thphe*(4.*np.cos(2.*lon)*np.sin(constA)-3.*np.sin(2.*lon)*np.cos(
        constA)) + 4.*np.cos(oblq)*np.sin(oblq)*beta21thphe*np.sin(lon)*np.sin(phase + constA))

    return thphe

@jit(nopython=True)
def modeththv(n, t, _lambda, sj, e, oblq, phase, colat, lon, hv, lv):

    beta20ththv = 0.75*(3.*hv - 10.*lv)*np.cos(2.*colat) + 0.75*(hv - 2.*lv)
    beta21ththv = 1.5*(3.*hv - 10.*lv)*np.sin(2.*colat)
    beta22ththv = -1.5*(3.*hv - 10.*lv)*np.cos(2.*colat) + 4.5*(hv - 2.*lv)

    ththv = (1./np.sqrt(1. + (-n/sj)*(-n/sj)))*(-6.*e*beta20ththv*np.cos(n*t - np.arctan(-n/sj) + np.arctan(_lambda)) + e*beta22ththv*(4.*np.sin(2.*lon)*np.sin(n*t - np.arctan(-n/sj) + np.arctan(_lambda)
                                                                                                                                                                )+3.*np.cos(2.*lon)*np.cos(n*t - np.arctan(-n/sj) + np.arctan(_lambda))) + 4.*np.cos(oblq)*np.sin(oblq)*beta21ththv*np.cos(lon)*np.sin(phase + n*t - np.arctan(-n/sj) + np.arctan(_lambda)))

    return ththv

@jit(nopython=True)
def modephphv(n, t, _lambda, sj, e, oblq, phase, colat, lon, hv, lv):

    beta20phphv = 0.75*(3.*hv - 8.*lv)*np.cos(2.*colat) + 0.75*(hv - 4.*lv)
    beta21phphv = 1.5*(3.*hv - 8.*lv)*np.sin(2.*colat)
    beta22phphv = -1.5*(3.*hv - 8.*lv)*np.cos(2.*colat) + 4.5*(hv - 4.*lv)

    phphv = (1./np.sqrt(1. + (-n/sj)*(-n/sj)))*(-6.*e*beta20phphv*np.cos(n*t - np.arctan(-n/sj) + np.arctan(_lambda)) + e*beta22phphv*(4.*np.sin(2.*lon)*np.sin(n*t - np.arctan(-n/sj) + np.arctan(_lambda)
                                                                                                                                                                )+3.*np.cos(2.*lon)*np.cos(n*t - np.arctan(-n/sj) + np.arctan(_lambda))) + 4.*np.cos(oblq)*np.sin(oblq)*beta21phphv*np.cos(lon)*np.sin(phase + n*t - np.arctan(-n/sj) + np.arctan(_lambda)))

    return phphv

@jit(nopython=True)
def modethphv(n, t, _lambda, sj, e, oblq, phase, colat, lon, hv, lv):

    beta21thphv = 3.*lv*np.sin(colat)
    beta22thphv = 3.*lv*np.cos(colat)

    thphv = (1./np.sqrt(1. + (-n/sj)*(-n/sj)))*(8.*e*beta22thphv*np.cos(2.*lon)*np.sin(n*t - np.arctan(-n/sj) + np.arctan(_lambda)) - 6.*e*beta22thphv*np.sin(2.*lon) *
                                                np.cos(n*t - np.arctan(-n/sj) + np.arctan(_lambda)) + 4.*np.cos(oblq)*np.sin(oblq)*beta21thphv*np.sin(lon)*np.sin(phase + n*t - np.arctan(-n/sj) + np.arctan(_lambda)))

    return thphv

@jit(nopython=True)
def stressththeNSR(constA_NSR, constB_NSR, colat, lon, he_NSR, le_NSR):
    alpha22ththe = -1.5*(3.*he_NSR - 10.*le_NSR) * \
        np.cos(2.*colat) + 4.5*(he_NSR - 2.*le_NSR)
    ththeNSR = constB_NSR*alpha22ththe*np.cos(2.*lon + constA_NSR)

    return ththeNSR

@jit(nopython=True)
def stressphpheNSR(constA_NSR, constB_NSR, colat, lon, he_NSR, le_NSR):

    alpha22phphe = -1.5*(3.*he_NSR - 8.*le_NSR) * \
        np.cos(2.*colat) + 4.5*(he_NSR - 4.*le_NSR)
    phpheNSR = constB_NSR*alpha22phphe*np.cos(2.*lon + constA_NSR)

    return phpheNSR

@jit(nopython=True)
def stressthpheNSR(constA_NSR, constB_NSR, colat, lon, he_NSR, le_NSR):

    alpha22thphe = 3.*le_NSR*np.cos(colat)
    thpheNSR = -2.*constB_NSR*alpha22thphe*np.sin(2.*lon + constA_NSR)

    return thpheNSR

@jit(nopython=True)
def modeththvNSR(constA_NSR, NSRrate, sj_NSR, colat, lon, hv_NSR, lv_NSR):

    alpha22ththv = -1.5*(3.*hv_NSR - 10.*lv_NSR) * \
        np.cos(2.*colat) + 4.5*(hv_NSR - 2.*lv_NSR)
    ththvNSR = (1./np.sqrt(1. + (-2.*NSRrate/sj_NSR)*(-2.*NSRrate/sj_NSR))) * \
        alpha22ththv*np.cos(2.*lon + constA_NSR -
                            np.arctan(-2.*NSRrate/sj_NSR))

    return ththvNSR

@jit(nopython=True)
def modephphvNSR(constA_NSR, NSRrate, sj_NSR, colat, lon, hv_NSR, lv_NSR):

    alpha22phphv = -1.5*(3.*hv_NSR - 8.*lv_NSR) * \
        np.cos(2.*colat) + 4.5*(hv_NSR - 4.*lv_NSR)
    phphvNSR = (1./np.sqrt(1. + (-2.*NSRrate/sj_NSR)*(-2.*NSRrate/sj_NSR))) * \
        alpha22phphv*np.cos(2.*lon + constA_NSR -
                            np.arctan(-2.*NSRrate/sj_NSR))

    return phphvNSR

@jit(nopython=True)
def modethphvNSR(constA_NSR, NSRrate, sj_NSR, colat, lon, hv_NSR, lv_NSR):

    alpha22thphv = 3.*lv_NSR*np.cos(colat)
    thphvNSR = -2.*(1./np.sqrt(1. + (-2.*NSRrate/sj_NSR)*(-2.*NSRrate/sj_NSR))) * \
        alpha22thphv*np.sin(2.*lon + constA_NSR -
                            np.arctan(-2.*NSRrate/sj_NSR))

    return thphvNSR

NameError: name 'jit' is not defined

In [3]:
def getStress(interior_value, e_in, colat, lon, steps, this_step, oblq, phase, NSRdelta):
    if isinstance(interior_value, str):
        interior = utils.import_interior(interior_value)
    else:
        interior = interior_value

    periodInSec = 306000.0  # sec;
    n = 2.*np.pi/periodInSec  # [rad/sec]

    t = (this_step/steps)*periodInSec

    _lambda = interior.rigidity / (interior.viscosity * n)

    if (e_in is None):
        ecc = interior.eccentricity
    else:
        ecc = e_in

    constA = n*t + np.arctan(_lambda)
    constB = 0.5*((n * n * interior.radius * interior.rigidity) /
                  interior.surface_gravity) * (1. / np.sqrt(1. + _lambda * _lambda))

    ththe = stressththe(constA, constB, ecc, oblq, phase,
                        colat, lon, interior.he, interior.le)
    phphe = stressphphe(constA, constB, ecc, oblq, phase,
                        colat, lon, interior.he, interior.le)
    thphe = stressthphe(constA, constB, ecc, oblq, phase,
                        colat, lon, interior.he, interior.le)

    if len(interior.modal_strengths) < 2:
        print("Number of modes is disallowed")
        return None
    ththv_vals = []
    phphv_vals = []
    thphv_vals = []

    for index in range(len(interior.modal_strengths)):
        ththv = modeththv(n, t, _lambda, interior.modal_strengths[index], ecc, oblq,
                          phase, colat, lon, interior.hv[index], interior.lv[index])
        phphv = modephphv(n, t, _lambda, interior.modal_strengths[index], ecc, oblq,
                          phase, colat, lon, interior.hv[index], interior.lv[index])
        thphv = modethphv(n, t, _lambda, interior.modal_strengths[index], ecc, oblq,
                          phase, colat, lon, interior.hv[index], interior.lv[index])

        ththv_vals.append(ththv)
        phphv_vals.append(phphv)
        thphv_vals.append(thphv)

    myStressThTh = ththe + constB * \
        np.sum(ththv_vals)
    myStressPhPh = phphe + constB * \
        np.sum(phphv_vals)
    myStressThPh = thphe + constB * \
        np.sum(thphv_vals)  # negative sign comes later

    # sum1 = np.sum(ththv_vals)
    # sum2 = np.sum(phphv_vals)
    # sum3 = np.sum(thphv_v)

    # COMPUTING NSR STRESSES FOR EUROPA

    if NSRdelta != 0:

        # RIGIDITY AND VISCOSITY were swapped in original version, now corrected and consistent with diurnal eqs above

        # delta of 0.1 for NSR period of 11419 years; 43 roughly equals 6 Myr period // 6/23: removed *year2sec
        NSRrate = (interior.rigidity / (2. * interior.viscosity * NSRdelta))

        constA_NSR = 2. * NSRrate * t + np.arctan(NSRdelta)
        #print("at constB NSR")
        constB_NSR = 0.5 * ((n * n * interior.radius * interior.rigidity) /
                            interior.surface_gravity) * (1. / np.sqrt(1. + NSRdelta * NSRdelta))

        # Currently sj values are from Hermes' paper for a reference model that is very similar to the visco model he sent us (M1)

        ththeNSR = stressththeNSR(
            constA_NSR, constB_NSR, colat, lon, interior.he_NSR, interior.le_NSR)
        phpheNSR = stressphpheNSR(
            constA_NSR, constB_NSR, colat, lon, interior.he_NSR, interior.le_NSR)
        thpheNSR = stressthpheNSR(
            constA_NSR, constB_NSR, colat, lon, interior.he_NSR, interior.le_NSR)

        if (len(interior.modal_strengths_nsr) < 2):
            print("Number of NSR modes is disallowed")
            return None

        ththvNSR_vals = []
        phphvNSR_vals = []
        thphvNSR_vals = []

        for index in range(len(interior.modal_strengths_nsr)):
            ththvNSR = modeththvNSR(
                constA_NSR, NSRrate, interior.modal_strengths_nsr[index], colat, lon,
                interior.hv_NSR[index], interior.lv_NSR[index])
            phphvNSR = modephphvNSR(
                constA_NSR, NSRrate, interior.modal_strengths_nsr[index], colat, lon,
                interior.hv_NSR[index], interior.lv_NSR[index])
            thphvNSR = modethphvNSR(
                constA_NSR, NSRrate, interior.modal_strengths_nsr[index], colat, lon,
                interior.hv_NSR[index], interior.lv_NSR[index])

            ththvNSR_vals.append(ththvNSR)
            phphvNSR_vals.append(phphvNSR)
            thphvNSR_vals.append(thphvNSR)

        # Combining stresses

        myStressThThNSR = ththeNSR + constB_NSR * \
            np.sum(ththvNSR_vals)
        myStressPhPhNSR = phpheNSR + constB_NSR * \
            np.sum(phphvNSR_vals)
        myStressThPhNSR = thpheNSR + constB_NSR * \
            np.sum(thphvNSR_vals)

        myStressThThTot = myStressThTh + myStressThThNSR
        myStressPhPhTot = myStressPhPh + myStressPhPhNSR
        myStressThPhTot = myStressThPh + myStressThPhNSR

    else:
        myStressThThTot = myStressThTh
        myStressPhPhTot = myStressPhPh
        myStressThPhTot = myStressThPh

    # Converting COMBINED STRESSES to principle stresses.

    if (myStressThThTot == myStressPhPhTot):
        zeta = np.pi/2
    else:
        # Amount by which coordinates are being rotated to get principal stresses; changes CCW, orientation of sigTheta
        zeta = 0.5*np.arctan((2.*myStressThPhTot) /
                             (myStressThThTot-myStressPhPhTot))

    sigTheta = myStressThThTot*(np.square(np.cos(zeta)))+myStressPhPhTot*(np.square(
        np.sin(zeta)))+myStressThPhTot*np.sin(2.*zeta)  # Corresponds to Terry's sigma 1
    sigPhi = myStressThThTot*(np.square(np.sin(zeta)))+myStressPhPhTot*(np.square(
        np.cos(zeta)))-myStressThPhTot*np.sin(2.*zeta)      # Corresponds to Terry's sigma 2

    sigThetaKPA = sigTheta*1E-3
    sigPhiKPA = sigPhi*1E-3

    zetaCW = (2.*np.pi)-zeta   # Changes to CW, still oriented along sigTheta

    # stress, The largest (i.e. most tensile) principal stress
    # heading, Direction of crack propagation/formation. Perpendicular to direction of largest principal stress (i.e. max tension).

    # Determines which stress is most tensile and largest heading.

    if (sigThetaKPA < sigPhiKPA):
        stress = sigPhiKPA         # In this case, sigPhi is the max stress, so the heading should be perpendicular to sigPhi. SigTheta is -| to sigPhi by definition, so heading = zetaCW
        heading = zetaCW
    else:
        stress = sigThetaKPA
        # In this case, sigTheta is the max stress, so the heading should be perpendicular to sigTheta. SigTheta is oriented along zeta, so heading = zetaCW + 90 deg
        heading = zetaCW + (np.pi/2.)

    # Making sure azimuth values fall between 0 and 360. WOULD LOVE TO CHANGE THIS TO MOD AND INCORPORATE ABOVE.
    if (heading >= (2.*np.pi)):
        # Also finds the two, complementary heading directions (e.g. 45 and 135)
        heading = heading - (2.*np.pi)
        heading2 = heading + np.pi
    else:
        heading2 = heading - np.pi

    # Determines which of the two heading directions (e.g. 45 or 135) is largest and selects that as the output heading.
    if (heading > heading2):
        bigHeading = heading
    else:
        bigHeading = heading2

    return (stress, bigHeading)


In [4]:
def stress2(interior, lon, lat, phase, tolerance, steps):
        results = []
        previous = 0
        for step in range(steps):
            current = getStress(
                interior_value=interior, 
                e_in=0.01, 
                colat=np.radians(90-lat), 
                lon=np.radians(360-lon), 
                steps=steps, 
                this_step=step,
                oblq=0.25,
                phase=np.radians(phase),
                NSRdelta=0)
            heading_degrees = np.degrees(current[1])
            results.append({
                'lon': lon,
                'lat': lat,
                'stress': current[0],
                'heading': heading_degrees,
                'headingCategory': utils.round_heading(heading_degrees, tolerance),
                'deltaStress': current[0] - previous,
                'time': step
            })
            previous = current[0]

        results[0]['deltaStress'] = results[0]['stress'] - results[-1]['stress']
        return results

In [5]:
def build_stress_field(interior, pointFrame, phase, tolerance=1, steps=360, is_async=True):
    stresses = []

    if is_async:
        pointStresses = Parallel(n_jobs=CPUS)(delayed(get_stresses_for_point)\
            (interior, point.lon, point.lat, phase, tolerance, steps) for point in pointFrame.itertuples())
    else:
        pointStresses = [get_stresses_for_point(interior, point.lon, point.lat, phase, tolerance, steps) for point in pointFrame.itertuples()]

    for stress in pointStresses:
        stresses.extend(stress)

    df = pd.DataFrame(stresses)

    return df

In [6]:
curve = fitArcs.fit_arc(arcs[0])


def test_perf():
    field = tools.get_stresses_for_point(interior, 60, 80, 200, 1, 360)
    field = stress2(interior, 60, 80, 200, 1, 360)

In [8]:
import cProfile
import pstats

cProfile.run('test_perf()', 'performance.log')

p = pstats.Stats('performance.log')
p.sort_stats('cumulative').print_stats(20)

Fri Apr 24 12:35:18 2020    performance.log

         54726 function calls in 0.138 seconds

   Ordered by: cumulative time
   List reduced from 34 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.138    0.138 {built-in method builtins.exec}
        1    0.000    0.000    0.138    0.138 <string>:1(<module>)
        1    0.000    0.000    0.138    0.138 <ipython-input-6-836475313489>:4(test_perf)
        1    0.004    0.004    0.069    0.069 /Users/stan/git/europa-cycloids/src/StressTools.py:165(get_stresses_for_point)
        1    0.004    0.004    0.068    0.068 <ipython-input-4-5295275ea32a>:1(stress2)
      360    0.049    0.000    0.064    0.000 /Users/stan/git/europa-cycloids/src/StressEquations.py:133(getStress)
      360    0.048    0.000    0.063    0.000 <ipython-input-3-fd0de786c14d>:1(getStress)
     2160    0.001    0.000    0.022    0.000 <__array_function__ internals>:2(sum)
     2160  

In [7]:
# get_stresses_for_point(interior, 60, 80, 200, 1, 360)

# MEW Tools

In [26]:
#from __future__ import division
import numpy as np
import sympy as sym
import math
#import time


r, θ, φ, t = sym.symbols(  'r θ φ t'    ,   real=True  )




############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
#                                 Love Number / y-Function Calculator                                      #
def lovefunc(L_, rin_, pin_, uin_, nin_, w_):                                                              #

    G = 6.67e-11


    # Number of layers in the body
    layno = len(rin_)


    # Find Purely fluid or elastic layers and create the index lists
    fluidlayerlist   = np.where(np.array(uin_) == 0)[0]
    elasticlayerlist = np.where(np.array(nin_) == 0)[0]


    # If there are no fluid layers, then create this list instead
    if len(fluidlayerlist) == 0:
        fluidlayerlist = [None]


    # Turn rigidity into viscoelastic parameter for those layers
    uV=[]
    for i in range(layno):
        if i in elasticlayerlist:
            uV.append(uin_[i])
        else:
            uV.append(uin_[i] / (1-1j*(uin_[i]/(nin_[i]*w_)) ))


    # Make a new lists of inputs
    rlist=rin_
    plist=pin_
    ulist=uV


    # Gravitational Acceleration Function
    def gfunc(r_, lay_):

        rsw0 = np.insert(rlist,0,0)

        gcalc = plist[lay_] * (r_**3 - rsw0[lay_]**3)

        for i in range(0,lay_):
            gcalc += plist[i] * (rsw0[i+1]**3 - rsw0[i]**3)

        return ((4*np.pi*G)/(3*r_**2)) * gcalc


    # Fundamental Matrix for Elastic Solutions
    def Emat(r_, g_, rho_, mu_):
        return sym.Matrix([
                           [                                             r_**(1+L_) ,                                r_**(-1+L_) , 0                      ,                                               r_**(-L_)   ,                                r_**(-2-L_) , 0                  ],
                           [ ((3+L_)/(L_*(1+L_)))                      * r_**(1+L_) , (1/L_)                       * r_**(-1+L_) , 0                      , -((-2+L_)/(L_*(1+L_)))                      * r_**(-L_)   , -(1/(1+L_))                  * r_**(-2-L_) , 0                  ],
                           [ (2*mu_*((-3+L_*(-1+L_))/L_) + r_*rho_*g_) * r_**(L_)   , (2*mu_*(-1+L_) + r_*rho_*g_) * r_**(-2+L_) , rho_     * r_**(L_)    , (2*mu_*((1-L_*(3+L_))/(1+L_)) + r_*rho_*g_) * r_**(-1-L_) , (2*mu_*(-2-L_) + r_*rho_*g_) * r_**(-3-L_) , rho_ * r_**(-1-L_) ],
                           [ 2*mu_*((2+L_)/(1+L_))                     * r_**(L_)   , 2*mu_*((-1+L_)/(L_))         * r_**(-2+L_) , 0                      , 2*mu_*((-1+L_)/(L_))                        * r_**(-1-L_) , 2*mu_*((2+L_)/(1+L_))        * r_**(-3-L_) , 0                  ],
                           [ 0                                                      , 0                                          ,            r_**(L_)    , 0                                                         , 0                                          ,        r_**(-1-L_) ],
                           [ 4*np.pi*G*rho_                           * r_**(1+L_) , 4*np.pi*G*rho_              * r_**(-1+L_) , (1+2*L_) * r_**(-1+L_) , 4*np.pi*G*rho_                             * r_**(-L_)   , 4*np.pi*G*rho_              * r_**(-2-L_) , 0                  ]
                           ])


    # Fundamental Matrix for Elastic Solutions Invserse
    def EmatINV(r_, g_, rho_, mu_):
        return sym.Matrix([
                           [ -(((2*mu_*(2+L_) - r_*rho_*g_)*L_*(1+L_))/(2*mu_*(3+4*L_*(2+L_))))         * r_**(-1-L_) , ((L_**2 *(1+L_)*(2+L_)) / (3+4*L_*(2+L_)))  * r_**(-1-L_) , -((L_*(1+L_))/(2*mu_*(3+4*L_*(2+L_)))) * r_**(-L_)  , ((L_**2 *(1+L_))/(2*mu_*(3+4*L_*(2+L_))))   * r_**(-L_)  , ((L_*(1+L_)*rho_)/(2*mu_*(3+4*L_*(2+L_))))  * r_**(-L_)  , 0                          ],
                           [  ((L_*(2*mu_*(L_*(3+L_)-1)-(1+L_)*r_*rho_*g_))/(2*mu_*(-1+4*L_**2)))       * r_**(1-L_)  , -((L_*(-1+L_)*(1+L_)**2)/(-1+4*L_**2))       * r_**(1-L_)  ,  ((L_*(1+L_))/(2*mu_*(-1+4*L_**2)))    * r_**(2-L_) , -(((-2+L_)*L_*(1+L_))/(2*mu_*(-1+4*L_**2))) * r_**(2-L_) , -((L_*(1+L_)*rho_)/(2*mu_*(-1+4*L_**2)))    * r_**(2-L_) , 0                          ],
                           [ -((4*np.pi*G*rho_)/(1+2*L_))                                              * r_**(1-L_)  , 0                                                         , 0                                                   , 0                                                        , 0                                                        , (1/(1+2*L_)) * r_**(1-L_)  ],
                           [  ((L_*(1+L_)*(2*mu_*(-1+L_) + r_*rho_*g_))/(2*mu_*(-1+4*L_**2)))           * r_**(L_)    , ((L_*(-1+L_)*(1+L_)**2)/(-1+4*L_**2))        * r_**(L_)    , -((L_*(1+L_))/(2*mu_*(-1+4*L_**2)))    * r_**(1+L_) , -((L_*(1+L_)**2)/(2*mu_*(-1+4*L_**2)))      * r_**(1+L_) , ((L_*(1+L_)*rho_)/(2*mu_*(-1+4*L_**2)))     * r_**(1+L_) , 0                          ],
                           [ -(((1+L_)*(2*mu_*(-3+L_*(-1+L_))+ L_*r_*rho_*g_))/(2*mu_*(3+4*L_*(2+L_)))) * r_**(2+L_)  , -((L_**2 *(1+L_)*(2+L_))/((3+4*L_*(2+L_)))) * r_**(2+L_)  , ((L_*(1+L_))/(2*mu_*(3+4*L_*(2+L_))))  * r_**(3+L_) , ((L_*(1+L_)*(3+L_))/(2*mu_*(3+4*L_*(2+L_)))) * r_**(3+L_) , -((L_*(1+L_)*rho_)/(2*mu_*(3+4*L_*(2+L_)))) * r_**(3+L_) , 0                          ],
                           [  ((4*np.pi*G*rho_)/(1+2*L_))                                              * r_**(2+L_)  , 0                                                         , 0                                                   , 0                                                        ,                                               r_**(1+L_) , -(1/(1+2*L_)) * r_**(2+L_) ]
                           ])


    # Fundamental Matrix for Fluid Solutions
    def Fmat(r_, g_, rho_):
        return sym.Matrix([
                           [ -(1/g_)                                              * r_**L_      , -(1/g_)                                               * r_**(-1-L_) ],
                           [ ((4*np.pi*G*rho_*r_ - (4+L_)*g_)/(L_*(1+L_)*g_**2)) * r_**L_      , ((4*np.pi*G*rho_*r_ + (-3+L_)*g_)/(L_*(1+L_)*g_**2)) * r_**(-1-L_) ],
                           [ 0                                                                  , 0                                                                   ],
                           [ 0                                                                  , 0                                                                   ],
                           [                                                        r_**L_      ,                                                         r_**(-1-L_) ],
                           [ ((-4*np.pi*G*rho_*r_ + (1+2*L_)*g_)/(g_))           * r_**(-1+L_) , -((4*np.pi*G*rho_)/(g_))                             * r_**(-1-L_) ]
                           ])


    # Fundamental Matrix for Fluid Solutions Inverse
    def FmatINV(r_, g_, rho_):
        return sym.Matrix([
                           [ ((4*np.pi*G*rho_)/((1+2*L_)*g_))                  * r_**(1-L_) ,  (1/(1+2*L_)) * r_**(1-L_) ],
                           [ (((1+2*L_)*g_ - 4*np.pi*G*rho_*r_)/((1+2*L_)*g_)) * r_**(1+L_) , -(1/(1+2*L_)) * r_**(2+L_) ]
                           ])


    # Make empty lists for boundary conditions resulting from elastic->fluid interface
    fluidBC     = []
    fluidBCvals = sym.Matrix(0, 1, [])


    # Make empty lists for each layer's fundimental matrix
    mats=[None]*(layno)


    # Determine if the core is fluid or not & produce core matrix
    if fluidlayerlist[0]==0:
        mats[0] = Fmat(r, gfunc(r, 0), plist[0])[:,0].applyfunc(sym.expand)
    else:
        mats[0] = Emat(r, gfunc(r, 0), plist[0], ulist[0])[:,0:3].applyfunc(sym.expand)


    # Produce the matrix for all the other layers
    for i in range(1,layno):
        if i-1 in fluidlayerlist and i in fluidlayerlist:

            mats[i] = (Fmat(r, gfunc(r, i), plist[i]) * FmatINV(rlist[i-1], gfunc(rlist[i-1], i-1), plist[i]) * mats[i-1][4:,:].subs(r,rlist[i-1])).applyfunc(sym.expand)
        #            print "fluid fluid"


        elif i-1 in fluidlayerlist and i not in fluidlayerlist:

            mattemp = mats[i-1][:,:].subs(r,rlist[i-1])
            mattemp[1:2,:]=sym.zeros(1,len(mattemp[1:2,:]))
            mattemp2 = sym.Matrix([[-1,0],[0,1],[-plist[i-1]*gfunc(rlist[i-1],i-1),0],[0,0],[0,0],[-4 *np.pi *G *plist[i-1],0]]).col_insert(0,mattemp)


            mats[i] = (Emat(r, gfunc(r, i), plist[i], ulist[i]) * EmatINV(rlist[i-1], gfunc(rlist[i-1], i-1), plist[i], ulist[i]) * mattemp2).applyfunc(sym.expand)
        #            print "fluid elastic"


        elif i-1 not in fluidlayerlist and i in fluidlayerlist:

            fluidBC.append((mats[i-1][2:3,:] - plist[i] * gfunc(rlist[i-1], i-1) * ((mats[i-1][4:5,:] / gfunc(rlist[i-1], i-1)) + mats[i-1][0:1,:])).subs(r,rlist[i-1]))
            fluidBC.append(mats[i-1][3:4,:].subs(r,rlist[i-1]))

            fluidBCvals = sym.Matrix([[0]]).row_insert(0,fluidBCvals)
            fluidBCvals = sym.Matrix([[0]]).row_insert(0,fluidBCvals)

            mats[i] = (Fmat(r, gfunc(r, i), plist[i]) * FmatINV(rlist[i-1], gfunc(rlist[i-1], i-1), plist[i]) * (mats[i-1][5:6,:] - 4*np.pi*G*plist[i]*((mats[i-1][4:5,:] / gfunc(rlist[i-1], i-1)) + mats[i-1][0:1,:])).subs(r,rlist[i-1]).row_insert(0,mats[i-1][4:5,:].subs(r,rlist[i-1]))).applyfunc(sym.expand)
        #            print "elastic fluid"


        elif i-1 not in fluidlayerlist and i not in fluidlayerlist:

            mats[i] = (Emat(r, gfunc(r, i), plist[i], ulist[i]) * EmatINV(rlist[i-1], gfunc(rlist[i-1], i-1), plist[i], ulist[i]) * mats[i-1].subs(r,rlist[i-1])).applyfunc(sym.expand)
    #            print "elastic elastic"



    # Give that matrix right at the body surface
    surfmat = mats[-1].subs(r,rlist[-1]).applyfunc(sym.expand)
    #    print "surface matrix"

    # Pad the list of elastic->fluid interface boundary condition with zeros to the right (so all rows have the same ammt of columns for BC constants)
    for i in range(len(fluidBC)):
        fluidBC[i] = sym.Matrix(np.lib.pad(fluidBC[i],((0,0),(0,len(surfmat[2:3,:])-len(fluidBC[i]))),'constant'))

    # Recast the e->f BC list as a matrix
    fluidBC = sym.Matrix(fluidBC)


    # Join the surface BCs to the e->f BCs
    if fluidlayerlist[-1]==layno-1:
        BCmat  = fluidBC.col_join(surfmat[5:6,:])
        BCvals = fluidBCvals.col_join(sym.Matrix([[-(2*L_+1)/rlist[-1]]]))
    elif fluidBC == sym.Matrix(0, 0, []):
        BCmat  = sym.Matrix([surfmat[2:3,:],surfmat[3:4,:],surfmat[5:6,:]])
        BCvals = sym.Matrix([[0],[0],[-(2*L_+1)/rlist[-1]]])
    else:
        BCmat  = fluidBC.col_join( sym.Matrix([surfmat[2:3,:],surfmat[3:4,:],surfmat[5:6,:]]))
        BCvals = fluidBCvals.col_join(sym.Matrix([[0],[0],[-(2*L_+1)/rlist[-1]]]))
    #    print "BC stuff"



    # Solve for the constants of integration
    consts = (BCmat.inv() * BCvals).applyfunc(sym.simplify)
    #    print "constants"




    # Calculate the Love numbers
    lovenums = [sym.expand(gfunc(rlist[-1], layno-1)*(surfmat[0:1,:]*consts)[0]) ,
                sym.expand(gfunc(rlist[-1], layno-1)*(surfmat[1:2,:]*consts)[0]) ,
                sym.expand(-((surfmat[4:5,:]*consts)[0] + 1)) ]
#    print "love numbers"



    # Calculate the y-functions in every layer
    yfuncs = []
    for i in range(len(mats)):
        yfuncs.append(sum((mats[i]*sym.Matrix(consts[:len(mats[i][0:1,:])])).applyfunc(sym.expand).tolist(),[]))
#    print "y functions"



    
    return lovenums


############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
#                                 Tide Calculator

#n, S, N, Ln, opn, t = sym.symbols('n S N Ln opn t',real=True)

def freqsinorder(n_, S_, N_, Ln_, opn_):
    return np.abs([ 2*n_ - 2*S_ - 2*N_ + 0*Ln_ + 0*opn_,
             1*n_ - 2*S_ - 2*N_ + 0*Ln_ + 0*opn_,
             3*n_ - 2*S_ - 2*N_ + 0*Ln_ + 0*opn_,
             0*n_ + 2*S_ + 2*N_ + 0*Ln_ - 2*opn_,
             1*n_ - 2*S_ - 2*N_ + 0*Ln_ + 2*opn_,
             1*n_ + 2*S_ + 2*N_ + 0*Ln_ - 2*opn_,
             2*n_ + 2*S_ + 2*N_ + 0*Ln_ - 4*opn_,
             3*n_ + 2*S_ + 2*N_ + 0*Ln_ - 4*opn_,
             1*n_ + 2*S_ + 2*N_ + 0*Ln_ - 4*opn_,
            -2*n_ + 2*S_ + 2*N_ + 1*Ln_ - 0*opn_,
             2*n_ - 2*S_ - 2*N_ + 1*Ln_ - 0*opn_,
            -2*n_ - 2*S_ - 2*N_ + 1*Ln_ + 4*opn_,
             0*n_ + 2*S_ + 2*N_ + 1*Ln_ - 2*opn_,
             0*n_ - 2*S_ - 2*N_ + 1*Ln_ + 2*opn_,
             2*n_ + 2*S_ + 2*N_ + 1*Ln_ - 4*opn_,
             2*n_ - 1*S_ - 1*N_ + 0*Ln_ - 1*opn_,
             3*n_ - 1*S_ - 1*N_ + 0*Ln_ - 1*opn_,
             1*n_ - 1*S_ - 1*N_ + 0*Ln_ - 1*opn_,
             0*n_ + 1*S_ + 1*N_ + 0*Ln_ - 1*opn_,
             1*n_ + 1*S_ + 1*N_ + 0*Ln_ - 1*opn_,
             1*n_ - 1*S_ - 1*N_ + 0*Ln_ + 1*opn_,
             2*n_ + 1*S_ + 1*N_ + 0*Ln_ - 3*opn_,
             3*n_ + 1*S_ + 1*N_ + 0*Ln_ - 3*opn_,
             1*n_ + 1*S_ + 1*N_ + 0*Ln_ - 3*opn_,
             2*n_ + 1*S_ + 1*N_ + 1*Ln_ - 3*opn_,
             0*n_ + 1*S_ + 1*N_ + 1*Ln_ - 1*opn_,
             2*n_ - 1*S_ - 1*N_ + 1*Ln_ - 1*opn_,
            -2*n_ + 1*S_ + 1*N_ + 1*Ln_ + 1*opn_,
             0*n_ - 1*S_ - 1*N_ + 1*Ln_ + 1*opn_,
            -2*n_ - 1*S_ - 1*N_ + 1*Ln_ + 3*opn_,
             1*n_ + 0*S_ + 0*N_ + 0*Ln_ + 0*opn_,
             2*n_ + 0*S_ + 0*N_ + 0*Ln_ - 2*opn_,
             1*n_ + 0*S_ + 0*N_ + 0*Ln_ - 2*opn_,
             3*n_ + 0*S_ + 0*N_ + 0*Ln_ - 2*opn_,
             0*n_ + 0*S_ + 0*N_ + 0*Ln_ + 0*opn_])


def potsinorder(G_, M_, R_, a_, e_, o_, op_, LA_, Lp_, n_, S_, N_, Ln_, opn_, t_ ,theta_, phi_):
    return [  ((3*G_*M_*R_**2)/(4*a_**3))   *              sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[0]*t_ - 2*phi_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[1]*t_ - 2*phi_),
             ((21*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[2]*t_ - 2*phi_),
              ((3*G_*M_*R_**2)/(8*a_**3))   *              sym.sin(o_)**2     *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[3]*t_ + 2*phi_ - 2*op_),
              ((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(o_)**2     *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[4]*t_ - 2*phi_ + 2*op_),
              ((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(o_)**2     *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[5]*t_ + 2*phi_ - 2*op_),
              ((3*G_*M_*R_**2)/(4*a_**3))   *              sym.sin(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[6]*t_ + 2*phi_ - 4*op_),
             ((21*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.sin(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[7]*t_ + 2*phi_ - 4*op_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ *         sym.sin(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[8]*t_ + 2*phi_ - 4*op_),
              ((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.cos(o_/2)**4   *                sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[9]*t_ + 2*phi_ + 0*op_ + Lp_),
             -((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.cos(o_/2)**4                *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[10]*t_ - 2*phi_ + 0*op_ + Lp_),
             -((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.sin(o_/2)**4                *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[11]*t_ - 2*phi_ + 4*op_ + Lp_),
              ((3*G_*M_*R_**2)/(8*a_**3))   * e_ * LA_ *   sym.sin(o_)**2                  *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[12]*t_ + 2*phi_ - 2*op_ + Lp_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ * LA_ *   sym.sin(o_)**2                  *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[13]*t_ - 2*phi_ + 2*op_ + Lp_),
              ((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.sin(o_/2)**4                *   sym.sin(theta_)**2                     *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[14]*t_ + 2*phi_ - 4*op_ + Lp_),
             -((3*G_*M_*R_**2)/(16*a_**3))  *              (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[15]*t_ - 1*phi_ - 1*op_),
             -((21*G_*M_*R_**2)/(32*a_**3)) * e_ *         (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[16]*t_ - 1*phi_ - 1*op_),
              ((3*G_*M_*R_**2)/(32*a_**3))  * e_ *         (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[17]*t_ - 1*phi_ - 1*op_),
             -((3*G_*M_*R_**2)/(8*a_**3))   *              sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[18]*t_ + 1*phi_ - 1*op_),
             -((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[19]*t_ + 1*phi_ - 1*op_),
              ((9*G_*M_*R_**2)/(16*a_**3))  * e_ *         sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[20]*t_ - 1*phi_ + 1*op_),
             -((3*G_*M_*R_**2)/(2*a_**3))   *              sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[21]*t_ + 1*phi_ - 3*op_),
             -((21*G_*M_*R_**2)/(4*a_**3))  * e_ *         sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[22]*t_ + 1*phi_ - 3*op_),
             ((3*G_*M_*R_**2)/(4*a_**3))    * e_ *         sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[23]*t_ + 1*phi_ - 3*op_),
             -((3*G_*M_*R_**2)/(4*a_**3))   * e_ * LA_ *   sym.sin(o_) * sym.sin(o_/2)**2  *   sym.sin(theta_) * sym.cos(theta_)      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[24]*t_ + 1*phi_ - 3*op_ + Lp_),
             -((3*G_*M_*R_**2)/(16*a_**3))   * e_ * LA_ *  sym.sin(2*o_)                  *   sym.sin(2*theta_)                       *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[25]*t_ + 1*phi_ - 1*op_ + Lp_),
              ((3*G_*M_*R_**2)/(32*a_**3))   * e_ * LA_ *  (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[26]*t_ - 1*phi_ - 1*op_ + Lp_),
              ((3*G_*M_*R_**2)/(32*a_**3))   * e_ * LA_ *  (2*sym.sin(o_) + sym.sin(2*o_)) *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[27]*t_ + 1*phi_ + 1*op_ + Lp_),
             -((3*G_*M_*R_**2)/(16*a_**3))   * e_ * LA_ *  sym.sin(2*o_)                   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[28]*t_ - 1*phi_ + 1*op_ + Lp_),
             -((3*G_*M_*R_**2)/(8*a_**3))   * e_ * LA_ *  sym.sin(o_) * sym.sin(o_/2)**2   *   sym.sin(2*theta_)                      *    sym.sin(freqsinorder(n_, S_, N_, Ln_, opn_)[29]*t_ - 1*phi_ + 3*op_ + Lp_),
             -((3*G_*M_*R_**2)/(32*a_**3))   * e_ *       (1 + 3*sym.cos(2*o_))            *   (1 + 3*sym.cos(2*theta_))              *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[30]*t_ ),
              ((3*G_*M_*R_**2)/(8*a_**3))   *             sym.sin(o_)**2                   *   (1 - 3*sym.cos(theta_)**2)             *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[31]*t_ - 2*op_),
              ((3*G_*M_*R_**2)/(32*a_**3))  * e_ *        sym.sin(o_)**2                   *   (1 + 3*sym.cos(2*theta_))              *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[32]*t_ - 2*op_),
             ((21*G_*M_*R_**2)/(16*a_**3))  * e_ *        sym.sin(o_)**2                   *   (1 - 3*sym.cos(theta_)**2)             *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[33]*t_ - 2*op_),
             -((G_*M_*R_**2)/(32*a_**3))    *             (1 + 3*sym.cos(2*o_))            *   (1 + 3*sym.cos(2*theta_))              *    sym.cos(freqsinorder(n_, S_, N_, Ln_, opn_)[34]*t_ )]


############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
############################################################################################################
#                                 Create the Satellite Class                                               #
class satellite:                                                                                           #

    def __init__(self,rin=[],pin=[],uin=[],nin=[],G=0,M=0,a=0,e=0,o=0,op=0,on=0,Ω=0,NSRn=0,libA=0,libp=0,libn=0):


        #All The Inputs
        self.rin = rin
        self.pin = pin
        self.uin = uin
        self.nin = nin

        self.G = G
        self.M = M
        self.a = a

        self.e = e



        self.o    = o
        self.op   = op
        self.on   = on


        self.NSRn    = NSRn


        self.libA   = libA
        self.libp   = libp
        self.libn   = libn


        # Body Radius
        self.R = self.rin[-1]

        # Body Mass
        self.mass = (4*np.pi/3) * pin[0]*rin[0]**3
        for i in range(1,len(rin)):
            self.mass += (4*np.pi/3) * pin[i]*(rin[i]**3 - rin[i-1]**3)

        # Mean Motion
        self.n = np.sqrt(self.G *(self.mass + self.M) / self.a**3)

        # grav accel
        self.g = self.G*self.mass / self.R**2


        if Ω == 0:
            self.Ω = self.n
        else:
            self.Ω = Ω



        self.freqlist = freqsinorder(self.n, self.Ω, self.NSRn, self.libn, self.on)
        self.potslist = potsinorder(self.G, self.M, self.R, self.a, self.e, self.o, self.op, self.libA, self.libp, self.n, self.Ω, self.NSRn, self.libn, self.on, t ,θ, φ)


        self.freqlist2 = []
        self.potslist2 = []
        self.static = []
        for i in range(len(self.freqlist)):
            if self.potslist[i] == 0:
                continue
            else:
                if self.freqlist[i] == 0:
                    self.static.append(self.potslist[i])
                else:
                    self.potslist2.append(self.potslist[i])
                    self.freqlist2.append(self.freqlist[i])

        self.static = [sum(self.static)]


        self.lovelist = []
        for i in self.freqlist2:
            self.lovelist.append(lovefunc(2, self.rin, self.pin, self.uin, self.nin, i))




        self.tt = 0
        self.pp = 0
        self.tp = 0
        for i in range(len(self.freqlist2)):

            #Surface Rigidity
            if self.nin[-1] == 0:
                self.uR = self.uin[-1]
            else:
                self.uR =  self.uin[-1] / (1-1j*(self.uin[-1]/(self.nin[-1]*self.freqlist2[i])))

            POT = self.potslist2[i]
            h = self.lovelist[i][0]
            l = self.lovelist[i][1]

            self.tt += ((2.0*self.uR)/(self.R*self.g)) * ((3.0*h -  6.0*l)*POT + l*sym.diff(POT,θ,θ))
            self.pp += ((2.0*self.uR)/(self.R*self.g)) * ((3.0*h - 12.0*l)*POT - l*sym.diff(POT,θ,θ))
            self.tp += ((2.0*self.uR)/(self.R*self.g)) *l * sym.csc(θ)*(sym.diff(POT,θ,φ) - sym.atan(θ)*sym.diff(POT,φ))

        self.ttR = sym.re(self.tt)
        self.ppR = sym.re(self.pp)
        self.tpR = sym.re(self.tp)

        self.PC1  = (1/2) * (self.ttR + self.ppR + sym.sqrt(4*self.tpR**2 + (self.ttR-self.ppR)**2))
        self.PC2  = (1/2) * (self.ttR + self.ppR - sym.sqrt(4*self.tpR**2 + (self.ttR-self.ppR)**2))
        self.PCΨ  = (1/2) * sym.atan( (2*self.tpR)/(self.ttR-self.ppR))
        self.PCΨ2 = (1/2) * sym.atan2((2*self.tpR),(self.ttR-self.ppR))



In [27]:
import yaml
import pathlib
import os

STRUCTURE_PATH = 'structures'
INTERIOR_PATH = 'interiors'

def import_structure(name, overrides={}):
    folder = './'
    path = os.path.join(folder, STRUCTURE_PATH, f'{name}.yml')
    
    with open(path, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except:
            config = {}

    config.update(overrides)

    radii = list(map(float, config['radii']))
    density = list(map(float, config['density']))
    rigidity = list(map(float, config['rigidity']))
    viscosity = list(map(float, config['viscosity']))

    G = 6.67e-11

    mass = float(config['mass']) if config.get('mass') is not None else 0
    semiMajorDistance = float(config['semiMajorDistance']) if config.get('semiMajorDistance') is not None else 0
    eccentricity = float(config['eccentricity']) if config.get('eccentricity') is not None else 0
    obliquity = float(config['obliquity']) if config.get('obliquity') is not None else 0
    obliquityPhase = float(config['obliquityPhase']) if config.get('obliquityPhase') is not None else 0
    obliquityPhaseRate = float(config['obliquityPhaseRate']) if config.get('obliquityPhaseRate') is not None else 0

    spinRate = float(config['spinRate']) if config.get('spinRate') is not None else 0

    nonSynchronusRotationRate = float(config['nonSynchronusRotationRate']) if config.get('nonSynchronusRotationRate') is not None else 0
    librationAmplitude = float(config['librationAmplitude']) if config.get('librationAmplitude') is not None else 0
    librationPhase = float(config['librationPhase']) if config.get('librationPhase') is not None else 0
    librationFrequency = float(config['librationFrequency']) if config.get('librationFrequency') is not None else 0

    sat = satellite(
        rin = radii,
        pin = density,
        uin = rigidity,
        nin = viscosity,
        G = G,
        M = mass,
        a = semiMajorDistance,
        e = eccentricity,
        o = obliquity,
        op = obliquityPhase,
        on = obliquityPhaseRate,
        Ω = spinRate,
        NSRn = nonSynchronusRotationRate,
        libA = librationAmplitude,
        libp = librationPhase,
        libn = librationFrequency
    )
    return sat

In [28]:
import numpy as np
import pandas as pd
import sympy as sym
import math
from sympy.printing.theanocode import theano_function
import matplotlib.pyplot as plt
import utils
import _MEWtools as mt
import cProfile
import pstats

def test_perf():
    import_structure('Sample')
    
cProfile.run('test_perf()', 'perf.log')

p = pstats.Stats('perf.log')
p.sort_stats('cumulative').print_stats(20)

Tue May 26 21:58:45 2020    perf.log

         14301448 function calls (13618798 primitive calls) in 7.231 seconds

   Ordered by: cumulative time
   List reduced from 982 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.231    7.231 {built-in method builtins.exec}
        1    0.000    0.000    7.231    7.231 <string>:1(<module>)
        1    0.000    0.000    7.231    7.231 <ipython-input-28-4a9aef62d789>:12(test_perf)
        1    0.000    0.000    7.231    7.231 <ipython-input-27-bd28f9dd2eca>:8(import_structure)
        1    0.000    0.000    7.224    7.224 <ipython-input-26-a80cf0d3b4af>:312(__init__)
        3    0.001    0.000    7.091    2.364 <ipython-input-26-a80cf0d3b4af>:20(lovefunc)
13443/8627    0.098    0.000    4.588    0.001 /Users/stan/Applications/Anaconda/anaconda3/envs/europa_cycloids/lib/python3.7/site-packages/sympy/core/operations.py:28(__new__)
23860/17579    0.022    0.000 

In [29]:
%timeit test_perf()

3.93 s ± 94.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
sat = import_structure('Sample')
f = sym.lambdify([t, φ, θ], sat.PC1)

f(1, 1, 1)

24971.07453460633